# Hyperparameter tuning momparison

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet50, ResNet50_Weights
import optuna
from tqdm import tqdm
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader, random_split

In [ ]:
# Just setting the same transforms as before 
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Dataset path
dataset_path = "data/breast_cancer"

# Load full dataset
dataset = ImageFolder(root=dataset_path, transform=transform)

# Split train/val/test (80/10/10)
total_size = len(dataset)
train_size = int(0.8 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size

train_ds, val_ds, test_ds = random_split(dataset, [train_size, val_size, test_size])

# Creating loaders
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [ ]:
def create_model(trial):
    base_model = resnet50(weights=ResNet50_Weights.DEFAULT)

    # Freezing
    for param in base_model.parameters():
        param.requires_grad = False

    # Tuning final classifier layers
    fc1_size = trial.suggest_int("fc1", 128, 512)
    dropout_rate = trial.suggest_float("dropout", 0.2, 0.5)

    base_model.fc = nn.Sequential(
        nn.Linear(base_model.fc.in_features, fc1_size),
        nn.ReLU(),
        nn.Dropout(dropout_rate),
        nn.Linear(fc1_size, 3)
    )

    return base_model.to(device)

In [ ]:
def objective(trial):
    model = create_model(trial)
    lr = trial.suggest_float("lr", 1e-5, 1e-3, log=True)
    optimizer = optim.Adam(model.fc.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    # Training 
    model.train()
    for epoch in range(3):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # Evaluation on validation set
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    val_accuracy = correct / total
    return 1.0 - val_accuracy  


In [11]:
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=20)

print("✅ Best Trial:")
print(f"  Value (1 - accuracy): {study.best_trial.value:.4f}")
print("  Params:")
for key, val in study.best_trial.params.items():
    print(f"    {key}: {val}")


[I 2025-03-28 12:05:51,217] A new study created in memory with name: no-name-d3b8e560-d654-44f9-8b87-84b478d0972a
[I 2025-03-28 12:08:34,797] Trial 0 finished with value: 0.2948717948717948 and parameters: {'fc1': 464, 'dropout': 0.45739396986948344, 'lr': 0.0008901668363214826}. Best is trial 0 with value: 0.2948717948717948.
[I 2025-03-28 12:11:12,026] Trial 1 finished with value: 0.3589743589743589 and parameters: {'fc1': 342, 'dropout': 0.4411962509383305, 'lr': 8.685966663377125e-05}. Best is trial 0 with value: 0.2948717948717948.
[I 2025-03-28 12:13:47,761] Trial 2 finished with value: 0.3589743589743589 and parameters: {'fc1': 189, 'dropout': 0.4184150202535579, 'lr': 9.241415756812604e-05}. Best is trial 0 with value: 0.2948717948717948.
[I 2025-03-28 12:16:27,669] Trial 3 finished with value: 0.3589743589743589 and parameters: {'fc1': 231, 'dropout': 0.36541043085385017, 'lr': 1.097892627580205e-05}. Best is trial 0 with value: 0.2948717948717948.
[I 2025-03-28 12:19:09,099] 

✅ Best Trial:
  Value (1 - accuracy): 0.2436
  Params:
    fc1: 402
    dropout: 0.32744624734009686
    lr: 0.0006964578043125617


In [ ]:
# Recreate and train best model using best trial params
best_params = study.best_trial.params

In [ ]:
# Build model using those params
def create_final_model(best_params):
    base_model = resnet50(weights=ResNet50_Weights.DEFAULT)
    for param in base_model.parameters():
        param.requires_grad = False

    base_model.fc = nn.Sequential(
        nn.Linear(base_model.fc.in_features, best_params["fc1"]),
        nn.ReLU(),
        nn.Dropout(best_params["dropout"]),
        nn.Linear(best_params["fc1"], 3)
    )
    return base_model.to(device)

In [14]:
# Train it again for full number of epochs
final_model = create_final_model(best_params)
optimizer = optim.Adam(final_model.fc.parameters(), lr=best_params["lr"])
criterion = nn.CrossEntropyLoss()

In [18]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_acc = correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}, Accuracy: {epoch_acc:.4f}")

    return model


In [19]:
trained_model = train_model(final_model, criterion, optimizer, train_loader, val_loader, num_epochs=10)


Epoch [1/10], Loss: 16.0731, Accuracy: 0.6410
Epoch [2/10], Loss: 14.0318, Accuracy: 0.6987
Epoch [3/10], Loss: 12.0669, Accuracy: 0.7756
Epoch [4/10], Loss: 10.8206, Accuracy: 0.7901
Epoch [5/10], Loss: 9.6208, Accuracy: 0.8189
Epoch [6/10], Loss: 8.8762, Accuracy: 0.8285
Epoch [7/10], Loss: 7.5168, Accuracy: 0.8942
Epoch [8/10], Loss: 7.5433, Accuracy: 0.8590
Epoch [9/10], Loss: 7.5465, Accuracy: 0.8590
Epoch [10/10], Loss: 7.0826, Accuracy: 0.8718


In [20]:
torch.save(trained_model.state_dict(), "best_model.pth")
